In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

columns = ['item ID','label','NR','text','na01','na02']
data = pd.read_csv("Sentiment Analysis Dataset.csv", names = columns, skiprows = 1,low_memory=False)
data.pop('na01')
data.pop('na02')
data.pop('item ID')
data.pop('NR')

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text.strip())
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

data_sample = data[0:20]

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf_sample = tfidf_vect.fit_transform(data_sample['text'])

X_features_sample = pd.concat([data_sample['body_len'], data_sample['punct%'], pd.DataFrame(X_tfidf_sample.toarray())], axis=1)
X_features_sample.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,85,86,87,88,89,90,91,92,93,94
0,34,38.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.57735,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27,11.1,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19,10.5,0.000000,0.0,0.000000,0.000000,0.600553,0.527895,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,97,12.4,0.249132,0.0,0.249132,0.249132,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30,13.3,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.1)

ValueError: Found input variables with inconsistent numbers of samples: [20, 1578614]

In [ ]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [ ]:
for n_est in[10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)